In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import datediff, avg
import pandas as pd

# Starting the Spark Session

In [ ]:
spark = SparkSession.builder.appName('dateml').getOrCreate()

# Loading the Data

In [ ]:
df = spark.read.csv('../tutorial-data/ClaimDataExample1.csv', header=True, inferSchema=True)

In [ ]:
df.printSchema()

# Registering the Table

In [ ]:
df.registerTempTable('ClaimMock')

In [ ]:
spark.catalog.listTables()

# Auto-selecting Date Fields

In [ ]:
date_columns = [i[0] for i in list(filter(lambda x: x[1]=='timestamp', df.dtypes))]
df.select(date_columns).show()

In [ ]:
df.withColumn('duration', datediff(df['RCV_DT'], df['SERV_DT'])).show()

# Feature Engineering on Date Fields

In [ ]:
# create all posible combinations
i = 0
while i < len(date_columns):    
    target_col = date_columns[i]
    for col in date_columns:
        if target_col != col:
            df = df.withColumn(f'{target_col}_to_{col}_diff', datediff(target_col, col))
    i += 1

In [ ]:
df.select(list(filter(lambda x: 'diff' in x, df.columns))).show()